<a href="https://colab.research.google.com/github/MMillward2012/deepmind_internship/blob/main/notebooks/1_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

x = torch.tensor([1, 2, 3], device=device)
print(x)
print(f"Running on device: {device}")


tensor([1, 2, 3], device='mps:0')
Running on device: mps


In [8]:
import pandas as pd
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification